<a href="https://colab.research.google.com/github/vfrantc/weather_experiments/blob/main/run_epdn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi 

Wed Jul 13 21:03:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    23W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
!cp /content/drive/MyDrive/hazy_detect/EPDN.zip .

In [4]:
!unzip EPDN.zip

Archive:  EPDN.zip
   creating: EPDN/
  inflating: EPDN/run_epdn.ipynb     
  inflating: EPDN/.gitignore         
  inflating: EPDN/Enhanced_Pix2Pix_Dehazing_Network.pdf  
  inflating: EPDN/precompute_feature_maps.py  
  inflating: EPDN/readme.md          
  inflating: EPDN/test.py            
  inflating: EPDN/train.py           
   creating: EPDN/.git/
  inflating: EPDN/.git/index         
   creating: EPDN/.git/info/
  inflating: EPDN/.git/info/exclude  
  inflating: EPDN/.git/description   
   creating: EPDN/.git/branches/
   creating: EPDN/.git/hooks/
  inflating: EPDN/.git/hooks/pre-push.sample  
  inflating: EPDN/.git/hooks/fsmonitor-watchman.sample  
  inflating: EPDN/.git/hooks/commit-msg.sample  
  inflating: EPDN/.git/hooks/pre-merge-commit.sample  
  inflating: EPDN/.git/hooks/applypatch-msg.sample  
  inflating: EPDN/.git/hooks/pre-commit.sample  
  inflating: EPDN/.git/hooks/post-update.sample  
  inflating: EPDN/.git/hooks/prepare-commit-msg.sample  
  inflating: EPDN/.g

In [5]:
%cd EPDN

/content/EPDN


In [6]:
!pip install dominate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
!ls /content/EPDN/cpt

16_net_G.pth  p2p


In [8]:
!cp /content/drive/MyDrive/weather_experiments/weather_test.zip .
!unzip weather_test.zip 

Streaming output truncated to the last 5000 lines.
  inflating: test/outdoor/input/0194_1_0.2.jpg  
  inflating: test/outdoor/input/1986_0.9_0.08.jpg  
  inflating: test/outdoor/input/0251_0.8_0.16.jpg  
  inflating: test/outdoor/input/1953_0.8_0.2.jpg  
  inflating: test/outdoor/input/0127_0.8_0.2.jpg  
  inflating: test/outdoor/input/0003_0.8_0.2.jpg  
  inflating: test/outdoor/input/1861_0.85_0.2.jpg  
  inflating: test/outdoor/input/0317_0.85_0.08.jpg  
  inflating: test/outdoor/input/0269_0.95_0.2.jpg  
  inflating: test/outdoor/input/1055_0.95_0.2.jpg  
  inflating: test/outdoor/input/0267_0.9_0.16.jpg  
  inflating: test/outdoor/input/1848_0.8_0.12.jpg  
  inflating: test/outdoor/input/0082_0.85_0.16.jpg  
  inflating: test/outdoor/input/0138_0.9_0.08.jpg  
  inflating: test/outdoor/input/1724_0.95_0.2.jpg  
  inflating: test/outdoor/input/1889_0.85_0.08.jpg  
  inflating: test/outdoor/input/0302_0.9_0.16.jpg  
  inflating: test/outdoor/input/1868_1_0.08.jpg  
  inflating: test/

In [10]:
!ls test

fog	     rain	     raindroptesta.txt	sand	    snowtest100k_L.txt
fog.txt      rain1400	     rain.txt		sand.txt    snow.txt
outdoor      rain1400.txt    real		snow	    test1
outdoor.txt  rain_drop_test  real.txt		Snow100K-L  test1.txt


In [9]:
import os
from glob import glob
from tqdm.notebook import tqdm 

In [11]:
!mv test/fog/input test/fog/mytest_A
!mv test/outdoor/input test/outdoor/mytest_A 
!mv test/real/input test/real/mytest_A 
!mv test/sand/input test/sand/mytest_A 
!mv test/test1/input test/test1/mytest_A

In [12]:
!mkdir /content/epdn

In [13]:
dataset = 'fog'
in_dir = '/content/EPDN/test/{}'.format(dataset)
model_path = '/content/EPDN/cpt'
!python3 /content/EPDN/test.py --which_epoch=16 --checkpoints_dir={model_path} --dataroot={in_dir}
!cp -r Pix2PixHD_modify/results/p2p/mytest_16/images /content/epdn/{dataset}

------------ Options -------------
aspect_ratio: 1.0
batchSize: 1
checkpoints_dir: /content/EPDN/cpt
cluster_path: features_clustered_010.npy
dataroot: /content/EPDN/test/fog
display_winsize: 512
feat_num: 3
fineSize: 512
gpu_ids: [0]
how_many: 4469
instance_feat: False
isTrain: False
label_feat: False
label_nc: 0
loadSize: 512
load_features: False
max_dataset_size: inf
model: pix2pixHD
nThreads: 2
n_blocks_global: 3
n_blocks_local: 3
n_clusters: 10
n_colors: 3
n_downsample_E: 4
n_downsample_global: 4
n_feats: 32
n_local_enhancers: 1
n_resblocks: 8
name: p2p
nef: 16
netG: local
ngf: 16
niter_fix_global: 0
no_flip: False
no_instance: true
norm: instance
ntest: inf
output_nc: 3
phase: mytest
resize_or_crop: none
results_dir: Pix2PixHD_modify/results/
rgb_range: 255
scale: [1]
serial_batches: False
tf_log: False
use_dropout: False
which_epoch: 16
-------------- End ----------------
CustomDatasetDataLoader
dataset [AlignedDataset] was created
------------ Options -------------
batchSize: 1

In [ ]:
!cp -r Pix2PixHD_modify/results/p2p/mytest_16/images . 

In [ ]:
for imname in glob('images/*.png'):
  new_name = "_".join(imname.split("_")[:-1]) + '.png'
  print(new_name)

images/YC_Bing_311.png
images/hv4_267.png
images/JL_Google_498.png
images/KRO_Google_490.png
images/hv22_28.png
images/GSGL_Bing_122.png
images/HF_Baidu_228.png
images/QC_Bing_312.png
images/TS_Baidu_486.png
images/PL_Google_839.png
images/hv11_528.png
images/XR_Baidu_276.png
images/XA_Bing_471.png
images/SGP_Google_244.png
images/CC_Bing_548.png
images/hv40_311.png
images/XR_Google_311.png
images/AM_Google_289.png
images/SJZ_Baidu_023.png
images/XR_Google_110.png
images/BJ_Google_455.png
images/SGP_Google_109.png
images/BJ_Google_409.png
images/hv35_24.png
images/hv47_1192.png
images/HazeDr_Google_709.png
images/BL_Google_127.png
images/GSGL_Baidu_1220.png
images/YD_Baidu_015.png
images/JL_Google_713.png
images/hv29_103.png
images/hv39_384.png
images/XA_Baidu_069.png
images/MSK_Bing_212.png
images/BJ_Bing_195.png
images/XR_Bing_385.png
images/hv5_50.png
images/SJZ_Baidu_127.png
images/HF_Baidu_122.png
images/GSGL_Google_382.png
images/PL_Google_394.png
images/NZL_Bing_017.png
images/G

In [ ]:
!mkdir out_epdn

In [ ]:
!ls RTTS

Annotations  ImageSets	mytest_A


In [ ]:
!mv images out_epdn/JPEGImages

In [ ]:
!cp /content/drive/MyDrive/new_test.json out_epdn/

In [ ]:
!tar czf out_epdn.tar.gz out_epdn/
!cp out_epdn.tar.gz /content/drive/MyDrive/hazy_detect/